In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

In [2]:
train=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [3]:
X_train_int=train[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

X_test_int=test[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
X_train_sc = X_train
X_test_sc = X_test

In [6]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [7]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np

def perform_svc_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # SVC hyperparameter candidate list setup
    param_grid = {
        'C': [0.1, 1, 10],
        'gamma': [1, 0.1, 0.01],
        'kernel': ['rbf', 'linear', 'poly'],
        'random_state': [0]
    }

    # Initialize an SVC model
    model = SVC()

    # Grid search setup
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # Fit the model and tune
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters
    print("Best Hyperparameters:", grid_search.best_params_)

    # Calculate the average evaluation metric
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
import numpy as np

def evaluate_svc_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # Initialize an SVC model
    model = SVC(**best_params, probability=True)

    # Lists to save the evaluation metrics for each fold
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # Fit the model
        model.fit(X_train_fold, y_train_fold)

        # Get the predicted probabilities on the test data
        probabilities = model.predict_proba(X_test_fold)

        # Adjust the predicted classes with a threshold of 0.4
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # Append the evaluation metrics for each fold to the lists
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # Save the model with the best F1 score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # Perform the final prediction with the model with the best F1 score
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # Calculate the evaluation metrics
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list




In [9]:
best_params, mean_f1_score = perform_svc_grid_search(X_train, y_train, k_fold=5)
evaluate_svc_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/Users/taewon/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/taewon/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/taewon/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/taewon/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d arr

Best Hyperparameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf', 'random_state': 0}
Mean F1 Score: 0.380069691769775
Fold 1
Accuracy: 0.821917808219178
Precision: 0.8571428571428571
Recall: 0.3333333333333333
F1 score: 0.48
Confusion Matrix:
[[54  1]
 [12  6]]
------------------------------
Fold 2
Accuracy: 0.7671232876712328
Precision: 0.5454545454545454
Recall: 0.3333333333333333
F1 score: 0.41379310344827586
Confusion Matrix:
[[50  5]
 [12  6]]
------------------------------
Fold 3
Accuracy: 0.8472222222222222
Precision: 0.8888888888888888
Recall: 0.4444444444444444
F1 score: 0.5925925925925926
Confusion Matrix:
[[53  1]
 [10  8]]
------------------------------
Fold 4
Accuracy: 0.8472222222222222
Precision: 0.7692307692307693
Recall: 0.5555555555555556
F1 score: 0.6451612903225806
Confusion Matrix:
[[51  3]
 [ 8 10]]
------------------------------
Fold 5
Accuracy: 0.8055555555555556
Precision: 0.75
Recall: 0.3333333333333333
F1 score: 0.46153846153846156
Confusion Matrix:
[[52  2]
 [

/Users/taewon/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_71281/3890911618.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
/var/folders/28/yz261nq91vq_lj47hynw_10r0000gn/T/ipykernel_71281/38909116

([0.821917808219178,
  0.7671232876712328,
  0.8472222222222222,
  0.8472222222222222,
  0.8055555555555556],
 [0.8571428571428571,
  0.5454545454545454,
  0.8888888888888888,
  0.7692307692307693,
  0.75],
 [0.3333333333333333,
  0.3333333333333333,
  0.4444444444444444,
  0.5555555555555556,
  0.3333333333333333],
 [0.48,
  0.41379310344827586,
  0.5925925925925926,
  0.6451612903225806,
  0.46153846153846156])